In [1]:
# Create the nodal network

import pypsa
import numpy as np
import pandas as pd

# Create network
network = pypsa.Network()

network.snapshots = [0,1]

network.add("Bus", 'Bus_1', v_nom=420, zone_nr=1, zone_name='NO1')
network.add("Bus", 'Bus_2', v_nom=420, zone_nr=2, zone_name='NO2')
network.add("Bus", 'Bus_3', v_nom=420, zone_nr=2, zone_name='NO2')

# Add transmission lines between each pair of buses
network.add("Line", "line_Bus_2_Bus_1", bus0="Bus_2", bus1="Bus_1", x=0.01, r=0.1, s_nom=10)
network.add("Line", "line_Bus_3_Bus_1", bus0="Bus_3", bus1="Bus_1", x=0.01, r=0.1, s_nom=7)
network.add("Line", "line_Bus_2_Bus_3", bus0="Bus_2", bus1="Bus_3", x=0.01, r=0.1, s_nom=10)

# Add load in bus 1. This is an uncertain load.
network.add("Load", 'load_1', bus='Bus_1', p_set=14)
network.add("Generator", 'expensive_gen', bus='Bus_1', p_set=0, p_nom=10, marginal_cost= 400, carrier = 'hydropower')

# Add generators in bus 2
network.add("Generator", 'cheap_gen', bus='Bus_2', p_set = 10, p_nom=10, marginal_cost= 100, carrier = 'hydropower')
network.add("Generator", 'medium_gen', bus='Bus_3', p_set = 4, p_nom=10, marginal_cost=200, carrier = 'hydropower')


# Add carriers
network.add('Carrier', ['hydropower', 'DC', 'AC', 'onshore-wind', 'offshore-wind'])
# Solve the network
network.pf(distribute_slack=True)

network.loads_t.p_set = network.loads_t.p

INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network <pypsa.components.SubNetwork object at 0x000001235513DBD0> for snapshots Index([0, 1], dtype='int64', name='snapshot')


In [3]:
from src.fbmc.network_conversion import nodal_to_zonal


zonal_network = nodal_to_zonal(network)
zonal_network.optimize(solver_name="gurobi")

ModuleNotFoundError: No module named 'src'

In [21]:
# Create Configuration
import src.fbmc.main as fbmc_main
import src.fbmc.config as fbmc_config

fbmc_config = fbmc_config.FBMCConfig()
fbmc_config.gsk_method = 'CURRENT_GENERATION'

# Calculate GSKs using different methods
from src.fbmc.config import FBMCConfig
import src.fbmc.parameters.gsk as gsk

# Method 0: Current Generation based GSK calculation
current_gen_config = FBMCConfig()
current_gen_config.gsk_method = "CURRENT_GENERATION"

# Method 1: Adjustable Capacity based GSK calculation
adjustable_config = FBMCConfig()
adjustable_config.gsk_method = "ADJUSTABLE_CAP"

# Method 2: Iterative Uncertainty based GSK calculation
uncertainty_config = FBMCConfig()
uncertainty_config.gsk_method = "ITERATIVE_UNCERTAINTY"
uncertainty_config.num_scenarios = 2
uncertainty_config.gen_variation_std_dev = 0.15
uncertainty_config.load_variation_std_dev = 0.1

# Method 3: Iterative FBMC based GSK calculation
fbmc_config = FBMCConfig()
fbmc_config.gsk_method = "ITERATIVE_FBMC"
fbmc_config.num_scenarios = 5
fbmc_config.max_gsk_iterations = 2
fbmc_config.gen_variation_std_dev = 0.15
fbmc_config.load_variation_std_dev = 0.1
fbmc_config.initial_gsk_method = "CURRENT_GENERATION"

# Get results for nodal network
print("GSK Calculation for Nodal Network:")
try:
    nodal_gsk_current_gen = gsk.calculate_gsk(network, current_gen_config)
    print("\nCurrent Generation method (t=0):")
    print(nodal_gsk_current_gen[0])
except Exception as e:
    print(f"Error with Current Generation method: {e}")

try:
    nodal_gsk_adjustable = gsk.calculate_gsk(network, adjustable_config)
    print("\nAdjustable Capacity method:")
    print(nodal_gsk_adjustable)
except Exception as e:
    print(f"Error with Adjustable Capacity method: {e}")

try:
    nodal_gsk_uncertainty = gsk.calculate_gsk(network, uncertainty_config)
    print("\nIterative Uncertainty method (t=0):")
    print(nodal_gsk_uncertainty[0])
except Exception as e:
    print(f"Error with Iterative Uncertainty method: {e}")

try:
    nodal_gsk_fbmc = gsk.calculate_gsk(network, fbmc_config)
    print("\nIterative FBMC method (t=0):")
    print(nodal_gsk_fbmc[0])
except Exception as e:
    print(f"Error with Iterative FBMC method: {e}")

GSK Calculation for Nodal Network:

Current Generation method (t=0):
     Bus_1     Bus_2     Bus_3
NO1    1.0  0.000000  0.000000
NO2    0.0  0.714286  0.285714

Adjustable Capacity method:
Bus  Bus_1  Bus_2  Bus_3
NO1    1.0    0.0    0.0
NO2    0.0    0.5    0.5

Iterative Uncertainty method (t=0):
     Bus_1  Bus_2  Bus_3
NO1    1.0    0.0    0.0
NO2    0.0    0.0    1.0
Calculating initial GSK using CURRENT_GENERATION method
Starting GSK iteration 1/2

Iterative Uncertainty method (t=0):
     Bus_1  Bus_2  Bus_3
NO1    1.0    0.0    0.0
NO2    0.0    0.0    1.0
Calculating initial GSK using CURRENT_GENERATION method
Starting GSK iteration 1/2
Starting GSK iteration 2/2
Starting GSK iteration 2/2
GSK converged after 2 iterations

Iterative FBMC method (t=0):
     Bus_1  Bus_2  Bus_3
NO1    1.0    0.0    0.0
NO2    0.0    0.0    1.0
GSK converged after 2 iterations

Iterative FBMC method (t=0):
     Bus_1  Bus_2  Bus_3
NO1    1.0    0.0    0.0
NO2    0.0    0.0    1.0


In [22]:
from src.fbmc import config
from src.fbmc.main import run_redispatch, setup_fbmc_model, calculate_fbmc_parameters, create_zonal_generation, add_fbmc_constraints, remove_original_constraints

network.pf()

# Calculate zonal network using current generation method
current_gen_config = FBMCConfig()
current_gen_config.gsk_method = "CURRENT_GENERATION"
current_gen_config.reliability_margin_factor = 0.0
zonal_current_gen_network = zonal_network.copy(snapshots=network.snapshots)

zonal_current_gen_network.optimize(solver_name="gurobi")

# Calculate FBMC parameters for the zonal network using current generation method
ram_current_gen, zPTDF_current_gen = calculate_fbmc_parameters(network, current_gen_config, nodal_gsk_current_gen)
zonal_current_gen_network = create_zonal_generation(zonal_current_gen_network)

GROWTH_FACTOR = (17/14)
# Add 20% extra load to the zonal network
zonal_current_gen_network.loads_t.p_set *= GROWTH_FACTOR
zonal_current_gen_network.loads_t.p *= GROWTH_FACTOR

zonal_current_gen_network = add_fbmc_constraints(zonal_current_gen_network, zPTDF_current_gen, ram_current_gen)
remove_original_constraints(zonal_current_gen_network)
zonal_current_gen_network.model.solve(solver_name="gurobi")

# Run redispatch
nodal_current_gen_rd = run_redispatch(network, zonal_current_gen_network)

# Calculate zonal network using adjustable capacity method
adjustable_config = FBMCConfig()
adjustable_config.gsk_method = "ADJUSTABLE_CAP"
adjustable_config.reliability_margin_factor = 0.0
zonal_adjustable_network = zonal_network.copy(snapshots=network.snapshots)

zonal_adjustable_network.optimize(solver_name="gurobi")


# Calculate FBMC parameters for the zonal network
ram_adjustable, zPTDF_adjustable = calculate_fbmc_parameters(network, adjustable_config, nodal_gsk_adjustable)
zonal_adjustable_network = create_zonal_generation(zonal_adjustable_network)

# # Add 20% extra load to the zonal network
zonal_adjustable_network.loads_t.p_set *= GROWTH_FACTOR
zonal_adjustable_network.loads_t.p *= GROWTH_FACTOR

zonal_adjustable_network = add_fbmc_constraints(zonal_adjustable_network, zPTDF_adjustable, ram_adjustable)
remove_original_constraints(zonal_adjustable_network)
zonal_adjustable_network.model.solve(solver_name="gurobi")

# Run redispatch
nodal_adjustable_rd = run_redispatch(network, zonal_adjustable_network)


# Calculate zonal network using iterative uncertainty method
iterative_config = FBMCConfig()
iterative_config.gsk_method = "ITERATIVE_UNCERTAINTY"
iterative_config.reliability_margin_factor = 0.0

zonal_iterative_network = zonal_network.copy(snapshots=network.snapshots)

zonal_iterative_network.optimize(solver_name="gurobi")

# Calculate FBMC parameters for the zonal network
ram_iterative, zPTDF_iterative = calculate_fbmc_parameters(network, iterative_config, nodal_gsk_uncertainty)
zonal_iterative_network = create_zonal_generation(zonal_iterative_network)

# # Add 20% extra load to the zonal network
zonal_iterative_network.loads_t.p_set *= GROWTH_FACTOR
zonal_iterative_network.loads_t.p *= GROWTH_FACTOR

zonal_iterative_network = add_fbmc_constraints(zonal_iterative_network, zPTDF_iterative, ram_iterative)
remove_original_constraints(zonal_iterative_network)
zonal_iterative_network.model.solve(solver_name="gurobi")

# Run redispatch
nodal_iterative_rd = run_redispatch(network, zonal_iterative_network)

# Calculate zonal network using iterative FBMC method
fbmc_iterative_config = FBMCConfig()
fbmc_iterative_config.gsk_method = "ITERATIVE_FBMC"
fbmc_iterative_config.reliability_margin_factor = 0.0
fbmc_iterative_config.num_scenarios = 5
fbmc_iterative_config.max_gsk_iterations = 2

zonal_fbmc_network = zonal_network.copy(snapshots=network.snapshots)

zonal_fbmc_network.optimize(solver_name="gurobi")

# Calculate FBMC parameters for the zonal network
ram_fbmc, zPTDF_fbmc = calculate_fbmc_parameters(network, fbmc_iterative_config, nodal_gsk_fbmc)
zonal_fbmc_network = create_zonal_generation(zonal_fbmc_network)

# # Add 20% extra load to the zonal network
zonal_fbmc_network.loads_t.p_set *= GROWTH_FACTOR
zonal_fbmc_network.loads_t.p *= GROWTH_FACTOR

zonal_fbmc_network = add_fbmc_constraints(zonal_fbmc_network, zPTDF_fbmc, ram_fbmc)
remove_original_constraints(zonal_fbmc_network)
zonal_fbmc_network.model.solve(solver_name="gurobi")

# Run redispatch
nodal_fbmc_rd = run_redispatch(network, zonal_fbmc_network)

INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.io: Writing time: 0.02s


INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.io: Writing time: 0.02s


Set parameter Username
Set parameter LicenseID to value 2594984
Academic license - for non-commercial use only - expires 2025-12-03
Read LP format model from file C:\Users\ameld\AppData\Local\Temp\linopy-problem-33ab9nss.lp
Reading time = 0.00 seconds
obj: 24 rows, 10 columns, 34 nonzeros
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 6800HS Creator Edition, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Set parameter LicenseID to value 2594984
Academic license - for non-commercial use only - expires 2025-12-03
Read LP format model from file C:\Users\ameld\AppData\Local\Temp\linopy-problem-33ab9nss.lp
Reading time = 0.00 seconds
obj: 24 rows, 10 columns, 34 nonzeros
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 6800HS Creator Edition, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logica

INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.io: Writing time: 0.02s


Set parameter Username
Set parameter LicenseID to value 2594984
Academic license - for non-commercial use only - expires 2025-12-03
Read LP format model from file C:\Users\ameld\AppData\Local\Temp\linopy-problem-33ab9nss.lp
Reading time = 0.00 seconds
obj: 24 rows, 10 columns, 34 nonzeros
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 6800HS Creator Edition, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Set parameter LicenseID to value 2594984
Academic license - for non-commercial use only - expires 2025-12-03
Read LP format model from file C:\Users\ameld\AppData\Local\Temp\linopy-problem-33ab9nss.lp
Reading time = 0.00 seconds
obj: 24 rows, 10 columns, 34 nonzeros
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 6800HS Creator Edition, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logica

...

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 10 primals, 24 duals
Objective: 3.60e+03
Solver model: available
Solver message: 2

INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.io: Writing time: 0.03s
INFO:linopy.io: Writing time: 0.03s


Set parameter Username
Set parameter LicenseID to value 2594984
Academic license - for non-commercial use only - expires 2025-12-03
Read LP format model from file C:\Users\ameld\AppData\Local\Temp\linopy-problem-tam34e02.lp
Reading time = 0.01 seconds
obj: 22 rows, 10 columns, 30 nonzeros
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 6800HS Creator Edition, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 22 rows, 10 columns and 30 nonzeros
Set parameter LicenseID to value 2594984
Academic license - for non-commercial use only - expires 2025-12-03
Read LP format model from file C:\Users\ameld\AppData\Local\Temp\linopy-problem-tam34e02.lp
Reading time = 0.01 seconds
obj: 22 rows, 10 columns, 30 nonzeros
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 6800HS Creator Edition, instruction se

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 10 primals, 22 duals
Objective: 5.40e+03
Solver model: available
Solver message: 2

INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.io: Writing time: 0.02s


Set parameter Username
Set parameter LicenseID to value 2594984
Academic license - for non-commercial use only - expires 2025-12-03
Read LP format model from file C:\Users\ameld\AppData\Local\Temp\linopy-problem-tzx5mpcg.lp
Reading time = 0.00 seconds
obj: 63 rows, 25 columns, 109 nonzeros
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 6800HS Creator Edition, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 63 rows, 25 columns and 109 nonzeros
Set parameter LicenseID to value 2594984
Academic license - for non-commercial use only - expires 2025-12-03
Read LP format model from file C:\Users\ameld\AppData\Local\Temp\linopy-problem-tzx5mpcg.lp
Reading time = 0.00 seconds
obj: 63 rows, 25 columns, 109 nonzeros
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 6800HS Creator Edition, instruction

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 25 primals, 63 duals
Objective: 2.27e-05
Solver model: available
Solver message: 2



In [28]:
zonal_current_gen_network.model.objective

Objective:
----------
LinearExpression: +400 Generator-p[0, expensive_gen] + 100 Generator-p[0, cheap_gen] + 200 Generator-p[0, medium_gen] + 400 Generator-p[1, expensive_gen] + 100 Generator-p[1, cheap_gen] + 200 Generator-p[1, medium_gen]
Sense: min
Value: 5066.663139467778

In [23]:
zonal_current_gen_network.model.solution['Generator-p']

<xarray.DataArray 'Generator-p' (snapshot: 2, Generator: 3)> Size: 48B
array([[ 0.66665785, 10.        ,  6.33334215],
       [ 0.66665785, 10.        ,  6.33334215]])
Coordinates:
  * snapshot   (snapshot) int64 16B 0 1
  * Generator  (Generator) object 24B 'expensive_gen' 'cheap_gen' 'medium_gen'

In [35]:
nodal_current_gen_rd.model.objective

Objective:
----------
LinearExpression: +1 Generator-p[0, expensive_gen_up] + 1 Generator-p[1, expensive_gen_up] + 1 Generator-p[0, expensive_gen_dn] ... +1 Generator-p[1, medium_gen_up] + 1 Generator-p[0, medium_gen_dn] + 1 Generator-p[1, medium_gen_dn]
Sense: min
Value: 3.333368605321999

In [33]:
nodal_current_gen_rd.model.solution['System-Cost']

<xarray.DataArray 'System-Cost' ()> Size: 8B
array(5400.)

In [29]:
zonal_fbmc_network.model.objective

Objective:
----------
LinearExpression: +400 Generator-p[0, expensive_gen] + 100 Generator-p[0, cheap_gen] + 200 Generator-p[0, medium_gen] + 400 Generator-p[1, expensive_gen] + 100 Generator-p[1, cheap_gen] + 200 Generator-p[1, medium_gen]
Sense: min
Value: 5399.9977325030995

In [24]:
zonal_fbmc_network.model.solution['Generator-p']

<xarray.DataArray 'Generator-p' (snapshot: 2, Generator: 3)> Size: 48B
array([[ 1.49999433, 10.        ,  5.50000567],
       [ 1.49999433, 10.        ,  5.50000567]])
Coordinates:
  * snapshot   (snapshot) int64 16B 0 1
  * Generator  (Generator) object 24B 'expensive_gen' 'cheap_gen' 'medium_gen'

In [34]:
nodal_fbmc_rd.model.solution['System-Cost']

<xarray.DataArray 'System-Cost' ()> Size: 8B
array(5400.)